In [ ]:
import urllib2, re
import sys,os
from apscheduler.schedulers.blocking import BlockingScheduler
import MySQLdb

In [ ]:
import logging
logging.basicConfig()

In [ ]:
from datetime import datetime

In [ ]:
class MessagePush():
    final = 0
    def getNews(self):
        src_url = 'http://www.trumba.com/calendars/seattle-city-council.rss'
        req = urllib2.Request(src_url)
        page = urllib2.urlopen(req).read()

        # source html file
        htmlFile = open('seattlegov.html','wb')
        htmlFile.write(page)
        htmlFile.close()

        p = re.compile(r'<title>(.*)</title>')
        p2 = re.compile(r'<description>(.*) &lt;br/&gt;&lt.*;Venue Location')
        
        t = p.findall(page)
        d = p2.findall(page)
        
        # clean data
        cancel = []
        titles = []
        committees = [] 
        for i in range(2,len(t)):
            if (t[i][0:8]=='CANCELED'):
                cancel.append("Yes")
            else :
                cancel.append("No")
            string = t[i].split("&#38;")
            titles.append(string[0])
            if (len(string)>1):
                committees.append(string[1])
            else:
                committees.append("")
        
        dates = []
        times = []
        weekdays = []
        locations = []
        
        for i in range(0,len(d)):
            s = d[i]
            s = " ".join(s.split(";br /&gt;"))
            s = ",".join(s.split(";br/&gt;"))
            s = "".join(s.split("&lt"))
            s = "".join(s.split("&gt;"))
            s = "".join(s.split("&amp;"))
            s = "".join(s.split("&quot;"))
            
            string = s.split(",")
            j = len(string)-1
            time = string[j]
            if time==' 2pm':
                time = '14:00'
            times.append(time)
            dateText = string[j-2][1:] + string[j-1]
            date = datetime.strptime(dateText,'%B %d %Y')
            dates.append(date)
            weekday = string[j-3]
            weekdays.append(weekday)
            if j-4 > 0:
                location = string[0]+string[1]
                locations.append(location)
            else:
                locations.append("")
            
        
        # store in mySQL
        conn = MySQLdb.connect(host='localhost',
                       user = 'root', 
                       passwd='560127910504',
                       db = 'DQD_TEST')
        x = conn.cursor()
        
        for i in range(len(titles)):
            agent = "Seattle City Council"
            title = titles[i]
            canceled = cancel[i]
            committee = committees[i]
            date = dates[i]
            time = times[i]
            weekday = weekdays[i]
            location = locations[i]
            self.final = self.final+1;
            
            try:
                x.execute("INSERT IGNORE INTO Meetings VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",\
                          (self.final,agent,title,canceled,committee,date,time,weekday,location,"WA 98124"))
                conn.commit()
            except:
                conn.rollback()
        x.close()  
 
        
# store data in a file        
        events = []
        for i in range(len(titles)):
           # item = titles[i]+' '+dates[i]+' '+times[i]+' '+zipcodes[i]+ ' '+locations[i]
            item = 'Seattle City Council '+titles[i]+times[i]+' '+locations[i]
            events.append(item)
        events.append(datetime.now().strftime("%I:%M %p on %B %d, %Y"))

        webFile = open('crawler_seattlegov.html','wb')
        for event in events:
            webFile.write(event)
            webFile.write('\n')
        webFile.close()


if __name__ == '__main__':
#     a = MessagePush()
#     print a.getNews()
    sched = BlockingScheduler()
    job_getNews = MessagePush().getNews
    sched.add_job(job_getNews, 'interval', minutes=1, id='job1')
    sched.start()
    
    # sched.remove_job('job1')
    # sched.shutdown()